Segmenting and Cluserting Neighborhoods in Toronoto: Week 3 Capstone Project

Import all the libraries that we will implement.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import geopy
from geopy.extra.rate_limiter import RateLimiter
import requests
import urllib.request, json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from bs4 import BeautifulSoup
import folium

PART ONE: Let's Scrape this website with a fork!

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = requests.get(url).text
sopa = BeautifulSoup(content, 'lxml')
wikitable = sopa.find('table', {'class': 'wikitable sortable'}).findAll('tr')

Let's extract the data!

In [62]:
raw = []

for row in wikitable:
    tds = row.find_all('td')
    if not tds:
        continue
    (postalcode, borough, neighborhood) = [td.text.strip() for td in tds]
    raw.append((postalcode, borough, neighborhood))

raw

[('M1A', 'Not assigned', 'Not assigned'),
 ('M2A', 'Not assigned', 'Not assigned'),
 ('M3A', 'North York', 'Parkwoods'),
 ('M4A', 'North York', 'Victoria Village'),
 ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'),
 ('M6A', 'North York', 'Lawrence Manor, Lawrence Heights'),
 ('M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"),
 ('M8A', 'Not assigned', 'Not assigned'),
 ('M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'),
 ('M1B', 'Scarborough', 'Malvern, Rouge'),
 ('M2B', 'Not assigned', 'Not assigned'),
 ('M3B', 'North York', 'Don Mills'),
 ('M4B', 'East York', 'Parkview Hill, Woodbine Gardens'),
 ('M5B', 'Downtown Toronto', 'Garden District, Ryerson'),
 ('M6B', 'North York', 'Glencairn'),
 ('M7B', 'Not assigned', 'Not assigned'),
 ('M8B', 'Not assigned', 'Not assigned'),
 ('M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'),
 ('M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'),
 ('M

Let mold this raw data into a polished DataFrame!

In [63]:
df = pd.DataFrame(raw, columns=['Postal Code', 'Borough', 'Neighborhood'])

df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Let's exterminate the 'Not assigned' object in Boroughs!

In [6]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

Set the Neighborhood name to Borough Name value is null!

In [64]:
change = df.Neighborhood == 'Not assigned'
df.loc[change, 'Neighborhood'] = df.loc[change, 'Borough']

df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [8]:
df = df.groupby('Postal Code').agg({'Borough': 'first',
                                    'Neighborhood': ', '.join}).reset_index()

Check out the Data!

In [9]:
df

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Cool, Cool, Cool, Everything looks Snazzy!

In [10]:
df.shape

(103, 3)

PART TWO: Let's Add Geogaphical cooridinates to the Postal Codes!

In [11]:
coord = pd.read_csv('https://cocl.us/Geospatial_data')
coord.rename(columns={'PostalCode': 'Postal Code'}, inplace=True)
coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Add coordinates to existing DataFrame

In [12]:
toro_comp_df = pd.merge(df, coord, on='Postal Code', how='inner')
toro_comp_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
toro_comp_df.shape

(103, 5)

PART THREE: Now let's start clustering them together. 
Gathering all the Four Square credentials.

In [15]:
CLIENT_ID = 'DMT1ZSCN0SBX05TSBCUAZUT2QH3K3215JXKBY11DKOW10K1F'
CLIENT_SECRET = 'X1GPLTRBWXHUMSA2MCFKCSIDPWXZUMJXUT2BHHPRTP1BEE3X'
LIMIT = 100
VERSION = '20201105'
radius = 500

Set the index with the Postal Code

In [16]:
toro_comp_df.set_index('Postal Code', inplace = True) 
neighbor_lat = toro_comp_df.loc['M5G']['Latitude']
neighbor_long = toro_comp_df.loc['M5G']['Longitude']

In [17]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, neighbor_lat, neighbor_long, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DMT1ZSCN0SBX05TSBCUAZUT2QH3K3215JXKBY11DKOW10K1F&client_secret=X1GPLTRBWXHUMSA2MCFKCSIDPWXZUMJXUT2BHHPRTP1BEE3X&v=20201105&ll=43.6579524,-79.3873826&radius=500&limit=100'

In [19]:
results = requests.get(url).json()

In [20]:
def venue_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']

In [22]:
near_venues = json_normalize(venues)
near_venues

<ipython-input-22-b4b920a8c87f>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  near_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id
0,e-0-537d4d6d498ec171ba22e7fe-0,0,"[{'summary': 'This spot is popular', 'type': '...",537d4d6d498ec171ba22e7fe,Jimmy's Coffee,82 Gerrard Street W,Gerrard & LaPlante,43.658421,-79.385613,"[{'label': 'display', 'lat': 43.65842123574496...",151,M5G 1Z4,CA,Toronto,ON,Canada,"[82 Gerrard Street W (Gerrard & LaPlante), Tor...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
1,e-0-4b744336f964a520d8d02de3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b744336f964a520d8d02de3,Somethin' 2 Talk About,78 Gerrard St W,NaN,43.658395,-79.385338,"[{'label': 'display', 'lat': 43.65839479027968...",171,M5G 1J5,CA,Toronto,ON,Canada,"[78 Gerrard St W, Toronto ON M5G 1J5, Canada]","[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",0,[],NaN,NaN
2,e-0-5a81ae339deb7d369fa7f146-2,0,"[{'summary': 'This spot is popular', 'type': '...",5a81ae339deb7d369fa7f146,Hailed Coffee,44 Gerrard St W,Yonge St,43.658833,-79.383684,"[{'label': 'display', 'lat': 43.65883296982352...",313,M5G,CA,Toronto,ON,Canada,"[44 Gerrard St W (Yonge St), Toronto ON M5G, C...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],College Park,NaN
3,e-0-557231e3498e540f05f3083c-3,0,"[{'summary': 'This spot is popular', 'type': '...",557231e3498e540f05f3083c,Tim Hortons,70 Gerrard St West,Bay St,43.658570,-79.385123,"[{'label': 'display', 'lat': 43.65856999999997...",194,M5G 1J5,CA,Toronto,ON,Canada,"[70 Gerrard St West (Bay St), Toronto ON M5G 1...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
4,e-0-5db70a0c306d6b000861ac9f-4,0,"[{'summary': 'This spot is popular', 'type': '...",5db70a0c306d6b000861ac9f,Neo Coffee Bar,770 Bay Street Unit 3,NaN,43.660140,-79.385870,"[{'label': 'display', 'lat': 43.66014, 'lng': ...",272,M5G 0A6,CA,Toronto,ON,Canada,"[770 Bay Street Unit 3, Toronto ON M5G 0A6, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
5,e-0-4a8355bff964a520d3fa1fe3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4a8355bff964a520d3fa1fe3,Mercatto,101 College St,NaN,43.660391,-79.387664,"[{'label': 'display', 'lat': 43.66039091189854...",272,M5G,CA,Toronto,ON,Canada,"[101 College St, Toronto ON M5G, Canada]","[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",0,[],NaN,NaN
6,e-0-4ab2b0b9f964a520e56b20e3-6,0,"[{'summary': 'This spot is popular', 'type': '...",4ab2b0b9f964a520e56b20e3,The Queen and Beaver Public House,35 Elm St.,at Bay St.,43.657472,-79.383524,"[{'label': 'display', 'lat': 43.65747228208784...",315,M5G 1H1,CA,Toronto,ON,Canada,"[35 Elm St. (at Bay St.), Toronto ON M5G 1H1, ...","[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",0,[],NaN,NaN
7,e-0-539c6f13498e06f4cc765165-7,0,"[{'summary': 'This spot is popular', 'type': '...",539c6f13498e06f4cc765165,The Elm Tree Restaurant,43 Elm St,Bay,43.657397,-79.383761,"[{'label': 'display', 'lat': 43.65739749535259...",298,M5G 1H1,CA,Toronto,ON,Canada,"[43 Elm St (Bay), Toronto ON M5G 1H1, Canada]","[{'id': '52e81612bcbc57f1066b79f9', 'name': 'M...",0,[],NaN,88534436
8,e-0-4c8facf91664b1f79c90aa2f-8,0,"[{'summary': 'This spot is popular', 'type': '...",4c8facf91664b1f79c90aa2f,College Park Area,College St.,NaN,43.659453,-79.383785,"[{'label': 'display', 'lat': 43.659453, 'lng':...",334,NaN,CA,Toronto,ON,Canada,"[College St., Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN
9,e-0-4e2284b11fc7c0ef9857d143-9,0,"[{'summary': 'This spot is popular', 'type': '...",4e2284b11fc7c0ef9857d143,Chatime 日出茶太,132 Dundas St W,btwn Bay & University,43.655542,-79.384684,"[{'label': 'dis

In [24]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
near_venues = near_venues.loc[:, filtered_columns]

near_venues['venue.categories'] = near_venues.apply(venue_type, axis=1)

near_venues.columns = [col.split(".")[-1] for col in near_venues.columns]

near_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,Tim Hortons,Coffee Shop,43.658570,-79.385123
4,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870


In [25]:
print('{} venues were returned by Foursquare.'.format(near_venues.shape[0]))

68 venues were returned by Foursquare.


In [29]:
def getVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    near_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    near_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 
                  'Venue Longitude', 'Venue Category']
    
    return(near_venues)

In [30]:
venues = getVenues(names=toro_comp_df['Borough'],latitudes=toro_comp_df['Latitude'],longitudes=toro_comp_df['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [32]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Group the venues by Neighborhood

In [33]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,104,104,104,104,104,104
Downtown Toronto,1248,1248,1248,1248,1248,1248
East Toronto,119,119,119,119,119,119
East York,79,79,79,79,79,79
Etobicoke,74,74,74,74,74,74
Mississauga,13,13,13,13,13,13
North York,240,240,240,240,240,240
Scarborough,89,89,89,89,89,89
West Toronto,153,153,153,153,153,153


Hot Coding the Venue Category

In [37]:
onehotcode = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehotcode['Neighborhood'] = venues['Neighborhood'] 

onehotcode.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [38]:
onehotcode.shape

(2139, 273)

In [39]:
group_neigh = onehot.groupby('Neighborhood').mean().reset_index()
group_neigh

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [40]:
group_neigh.shape

(10, 273)

In [43]:
max_top_v = 5

for hood in group_neigh['Neighborhood']:
    print("----"+hood+"----")
    temp = group_neigh[group_neigh['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(max_top_v))  

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Park  0.06
3            Café  0.06
4     Pizza Place  0.05
----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.11
1                Café  0.05
2          Restaurant  0.04
3               Hotel  0.03
4  Seafood Restaurant  0.02
----East Toronto----
                venue  freq
0         Coffee Shop  0.07
1    Greek Restaurant  0.06
2  Italian Restaurant  0.04
3             Brewery  0.04
4              Bakery  0.03
----East York----
            venue  freq
0    Intersection  0.05
1     Coffee Shop  0.05
2            Bank  0.05
3  Sandwich Place  0.04
4            Park  0.04
----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.07
2  Sandwich Place  0.07
3        Pharmacy  0.05
4             Gym  0.04
----Mississauga----
                      venue  freq
0                     Hotel  0.15
1               Coffee Shop  0.15
2   

In [44]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
number_top = 10

indict = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(number_top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indict[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighbor_venues_sort = pd.DataFrame(columns=columns)
neighbor_venues_sort['Neighborhood'] = group_neigh['Neighborhood']

for ind in np.arange(group_neigh.shape[0]):
    neighbor_venues_sort.iloc[ind, 1:] = most_common_venues(group_neigh.iloc[ind, :], number_top)

neighbor_venues_sort

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Restaurant,Bagel Shop,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Bakery,Park,Italian Restaurant,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Italian Restaurant,Brewery,Park,Ice Cream Shop,Restaurant,Bookstore,Café,Pub
3,East York,Intersection,Coffee Shop,Bank,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Indian Restaurant,Pet Store
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Fast Food Restaurant,Grocery Store,Gym,Liquor Store,Café,Convenience Store
5,Mississauga,Coffee Shop,Hotel,Middle Eastern Restaurant,Gym,Intersection,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,American Restaurant,Gas Station
6,North York,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Grocery Store,Restaurant,Bank,Park,Fast Food Restaurant
7,Scarborough,Bakery,Coffee Shop,Bank,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Indian Restaurant,Intersection,Breakfast Spot,Smoke Shop
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Pizza Place,Gift Shop,Grocery Store
9,York,Park,Breakfast Spot,Hockey Arena,Bar,Dog Run,Bus Line,Pool,Tennis Court,Sandwich Place,Discount Store


In [54]:
kclusters = 5

grouped_clustering = group_neigh.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
kmeans.labels_[0:10]

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1])

In [59]:
merged = group_neigh

merged['Cluster Labels'] = kmeans.labels_
merged = merged.join(neighbor_venues_sort.set_index('Neighborhood'), on='Neighborhood')
merged.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venu

Mapping it Out!

In [ ]:
toronto_geo = Nominatim(user_agent="Coursera")
address = 'Toronto'
try:
    location = toronto_geo.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

toro_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toro_comp_df['Latitude'], toro_comp_df['Longitude'], toro_comp_df['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toro_map)  
    
toro_map